自动求导：自动微分
PyTorch使用了一种叫做自动微分的技术。也就是说，它会有一个记录我们所有执行操作的记录器，之后再回放记录来计算我们的梯度。这一技术在构建神经网络时尤其有效，因为我们可以通过计算前路参数的微分来节省时间。

在pytorch中所有神经网络的中心，就是autograd包。让我们先快速的访问一下它，然后训练我们自己的第一个神经网络。

autograd包为所有在tensor上的的操作提供自动微分。它是一个运行时刻定义的框架，这意味着你的反向传播(backprop)是在你的代码运行时候定义的，取决于你代码怎么运行，并且每次迭代都是不同的。

tensor
torch.tensor是自动求导的中心类。如果你把它的属性.requires_grad设置成true，它将开始追踪所有在它上面的操作。当你完成你所有的计算之后，你可以调用.backward()方法来把所有的梯度自动计算一遍。此tensor的梯度将会累积到它的.grad属性里面。

如果你要停止一个tensor的历史追踪，你可以调用.detach()方法来把计算历史进行分离开来，并且阻止后续的计算被追踪。
